Setup para cuaderno

In [1]:
%set_env GOOGLE_APPLICATION_CREDENTIALS=google-credentials.json

env: GOOGLE_APPLICATION_CREDENTIALS=google-credentials.json


In [5]:
import DB_Control
import requests, json
from bs4 import BeautifulSoup
from StockMarket.market import StockMarket

db = DB_Control.initialise_database()
stonks = StockMarket(db)

	Inicializando submódulo StockMarket.market
	Módulo cargado con éxito


Recrear archivo dump con información de las acciones

In [3]:
stonks_to_search = ['AAPL','ABT','AMC','AMD','AMZN','AXP','BABA','BB','CVX','DIS','F','FB','GE','GM','GME','GOOGL','HPQ','IBM','JNJ','KO','MCD','MMM','MSFT','NFLX','NOK','NVDA','ORCL','PEP','PFE','PYPL','SBUX','SNE','T','TSLA','TWTR','V','VZ','WMT','XOM','ZM','TSM','EBAY','TCEHY','BA','XIACF','BRK.B','CRM','UBER','SIRI','LUV','DAL','WORK','CRM','ET','RCL','ZNGA','NKLA','JBLU','GPRO','ZOM','AMC','TXMD','SAVE',]
dump = {}
for stonk in stonks_to_search:
  page = requests.get(f"https://www.stockwatch.com/Quote/Detail?U:{stonk}")
  soup = BeautifulSoup(page.content, "html.parser")
  print(f"Revisando acción: {stonk}")
  
  header = soup.find('td')
  stock_info = header.text.split(" - ")
  stock_data = soup.find('tr', class_=f"Upd").find_all("td")
  name = stock_info[1]
  website = stock_info[2]
  if stock_info[0][0] == 'Q':
    market = "NASDAQ"
  else:
    market = "NYSE"
  try:
    stock_amt = int(float("".join(stock_data[9].text.split(","))) * 1000)
    cap = float("".join(stock_data[10].text.split(","))) * 1000
  except ValueError:
    stock_amt = int(float("".join(stock_data[6].text.split(","))) * 1000)
    cap = float("".join(stock_data[7].text.split(","))) * 1000
  pps = cap/stock_amt

  
  dump[stonk] = {'market': market,
                 'name': name,
                 'website': website,
                 'market_cap': cap,
                 'per_stock_price': pps,
                 'vol': stock_amt,
                 'cats': []
                }
with open('dump.json', 'w') as f:
  print(json.dumps(dump, indent=4), file=f)
  print("dump.json generado satisfactoriamente")

Revisando acción: AAPL
Revisando acción: ABT
Revisando acción: AMC
Revisando acción: AMD
Revisando acción: AMZN
Revisando acción: AXP
Revisando acción: BABA
Revisando acción: BB
Revisando acción: CVX
Revisando acción: DIS
Revisando acción: F
Revisando acción: FB
Revisando acción: GE
Revisando acción: GM
Revisando acción: GME
Revisando acción: GOOGL
Revisando acción: HPQ
Revisando acción: IBM
Revisando acción: JNJ
Revisando acción: KO
Revisando acción: MCD
Revisando acción: MMM
Revisando acción: MSFT
Revisando acción: NFLX
Revisando acción: NOK
Revisando acción: NVDA
Revisando acción: ORCL
Revisando acción: PEP
Revisando acción: PFE
Revisando acción: PYPL
Revisando acción: SBUX
Revisando acción: SNE
Revisando acción: T
Revisando acción: TSLA
Revisando acción: TWTR
Revisando acción: V
Revisando acción: VZ
Revisando acción: WMT
Revisando acción: XOM
Revisando acción: ZM
Revisando acción: TSM
Revisando acción: EBAY
Revisando acción: TCEHY
Revisando acción: BA
Revisando acción: XIACF
Revisa

Convertir archivo dump en acciones en google firebase

In [7]:
with open("dump.json") as file:
    dump = json.load(file)
    for symbol, info in dump.items():
        payload = {
            "name": info['name'],
            "market": info['market'],
            "website": info['website'],
            "total_stock_amount": info["vol"],
            "per_stock_price": round(info["per_stock_price"], 2),
            "categories": [],
            "unique_events": [],
            "current_stock_amount": info["vol"]
        }
        stonks.setup_stock(symbol, payload)

In [8]:
for doc in stonks.stock_refs.values():
    docs = doc.collection("price_history").list_documents()
    for file in docs:
        file.delete()

Unknown: None Stream removed